<a href="https://colab.research.google.com/github/akalpit23/Explainable-techniques/blob/main/Assignment-7/Explainable_Deep_Learning_Assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #07



## Akalpit Dawkhar

### **Explainable_Deep_Learning**

### Overview

In [1]:
# Please use this to connect your GitHub repository to your Google Colab notebook
# Connects to any needed files from GitHub and Google Drive
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "Explainable-techniques/Assignment-7" # Change to your repo name
git_path = 'https://github.com/akalpit23/Explainable-techniques.git' #Change to your path
!git clone "{git_path}"

# Install dependencies from requirements.txt file
!pip install -q -r "{os.path.join(repo_name,'requirements.txt')}" #Add if using requirements.txt

# Change working directory to location of notebook
notebook_dir = ''
path_to_notebook = os.path.join(repo_name,notebook_dir)
%cd "/content/Explainable-techniques/Assignment-7"
%ls

Cloning into 'Explainable-techniques'...
remote: Enumerating objects: 233, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 233 (delta 18), reused 24 (delta 9), pack-reused 199 (from 1)
Receiving objects: 100% (233/233), 8.01 MiB | 12.97 MiB/s, done.
Resolving deltas: 100% (119/119), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.0/37.0 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 48.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build whee

# Working with TCAV

In [2]:
# We are going to clone the tcav repo in GitHub:
!git clone https://github.com/tensorflow/tcav.git tcav
%cd tcav
!ls

Cloning into 'tcav'...
remote: Enumerating objects: 709, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 709 (delta 125), reused 111 (delta 111), pack-reused 570 (from 1)
Receiving objects: 100% (709/709), 617.05 KiB | 4.67 MiB/s, done.
Resolving deltas: 100% (419/419), done.
/content/Explainable-techniques/Assignment-7/tcav
CONTRIBUTING.md        LICENSE	  requirements.txt  Run_TCAV_on_colab.ipynb  tcav
FetchDataAndModels.sh  README.md  Run_TCAV.ipynb    setup.py


### Creating a monkey patch to change classes and broden concepts

In [3]:
import importlib.util

import sys
sys.path.append('/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet')

# Define the path to the file that you want to import
module_path = "/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet/download_and_make_datasets.py"


# Load the module
spec = importlib.util.spec_from_file_location("download_and_make_datasets", module_path)
original_script = importlib.util.module_from_spec(spec)
spec.loader.exec_module(original_script)

# Define the new function with the updated classes and concepts
def make_concepts_targets_and_randoms_patched(source_dir, number_of_images_per_folder, number_of_random_folders):
    import subprocess
    import os
    from tensorflow.io import gfile
    import imagenet_and_broden_fetcher as fetcher

    # Run script to download data to source_dir
    if not gfile.exists(source_dir):
        gfile.makedirs(source_dir)
    if not gfile.exists(os.path.join(source_dir, 'broden1_224/')) or not gfile.exists(os.path.join(source_dir, 'inception5h')):
        subprocess.call(['bash', 'FetchDataAndModels.sh', source_dir])

    # Patch: Change the classes and concepts
    imagenet_classes = ['leopard']  # Changed from 'zebra' to 'eagle'
    broden_concepts = ['dotted', 'perforated', 'banded']  # Changed from 'striped', 'dotted', 'zigzagged'

    # Fetch targets from ImageNet
    imagenet_dataframe = fetcher.make_imagenet_dataframe("/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet/imagenet_url_map.csv")
    for image in imagenet_classes:
        fetcher.fetch_imagenet_class(source_dir, image, number_of_images_per_folder, imagenet_dataframe)

    # Fetch concepts from Broden
    for concept in broden_concepts:
        fetcher.download_texture_to_working_folder(broden_path=os.path.join(source_dir, 'broden1_224'),
                                                   saving_path=source_dir,
                                                   texture_name=concept,
                                                   number_of_images=number_of_images_per_folder)

    # Create random folders for TCAV experiments
    fetcher.generate_random_folders(
        working_directory=source_dir,
        random_folder_prefix="random500",
        number_of_random_folders=number_of_random_folders + 1,
        number_of_examples_per_folder=number_of_images_per_folder,
        imagenet_dataframe=imagenet_dataframe
    )

# Override the original function with the patched version
original_script.make_concepts_targets_and_randoms = make_concepts_targets_and_randoms_patched


# Now you can run the patched function
if __name__ == '__main__':
    source_dir = "/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER"  # Specify your directory
    number_of_images_per_folder = 10  # Number of images per class or concept
    number_of_random_folders = 10  # Number of random folders for TCAV

    original_script.make_concepts_targets_and_randoms(
        source_dir, number_of_images_per_folder, number_of_random_folders
    )


Downloaded 10 for leopard


In [4]:
# Next, we will create the datasets
%cd /content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet
%run download_and_make_datasets.py --source_dir=YOUR_FOLDER --number_of_images_per_folder=10 --number_of_random_folders=10

%cd /content/tcav

/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet
Downloaded 10 for zebra
Successfully created data at YOUR_FOLDER
[Errno 2] No such file or directory: '/content/tcav'
/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet


# Hypothesis:

Null Hypothesis (H0): The ResNet50 model pre-trained on ImageNet does not show a significant difference in its ability to learn the concept "dotted" compared to the concepts "striped" and "banded" for the class 'leopard'.

Alternative Hypothesis (H1): The ResNet50 model pre-trained on ImageNet shows a significant difference in its ability to learn the concept "dotted" compared to the concepts "striped" and "banded" for the class 'leopard', specifically, it learns the concept "dotted" but not the concepts "striped" or "banded".

In [5]:
!pip install tcav matplotlib==2.2.4 Pillow==8.2.0 scikit-learn==0.20.3 scipy==1.2.1 tensorflow==2.14.1 numpy==1.19.2 protobuf==3.10.0 pandas==1.0.3

  Using cached matplotlib-2.2.4.tar.gz (37.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached Pillow-8.2.0.tar.gz (47.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached scikit-learn-0.20.3.tar.gz (11.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached scipy-1.2.1.tar.gz (23.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached tensorflow-2.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached numpy-1.19.2.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.



In [6]:
# Basic
import os
import absl
import random
import matplotlib.pyplot as plt
from PIL import Image

# Models
import tensorflow as tf

#XAI
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.model  as model
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot

ModuleNotFoundError: No module named 'tcav'

In [ ]:
# Model wrapper (InceptionV3 and GoogleNet are provided in model.py)
model_to_run = 'GoogleNet'
# Name of the parent directory that results are stored (if cacheing)
project_name = 'tcav_class_test'
working_dir = '/content/tcav/tcav'
# Location where activations are stored
activation_dir =  working_dir+ '/activations/'
# Location where CAVs are stored (or None if not storing)
cav_dir = working_dir + '/cavs/'
# Location of images
source_dir = '/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER'
# Bottleneck layer
bottlenecks = ['mixed4c']

# Create directories if they don't exist
utils.make_dir_if_not_exists(activation_dir)
utils.make_dir_if_not_exists(working_dir)
utils.make_dir_if_not_exists(cav_dir)

# Regularizer penalty parameter for linear classifier to get CAVs
alphas = [0.1]

target = 'leopard'
concepts = ["dotted","striped","banded"]

In [ ]:
%cp -av '/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/mobilenet_v2_1.0_224' '/content/tcav/tcav/mobilenet_v2_1.0_224'
%rm '/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/mobilenet_v2_1.0_224'

In [ ]:
%cp -av '/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h' '/content/tcav/tcav/inception5h'
%rm '/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h

In [ ]:
def display_random_images(directory, num_images):
    # Get list of files in the directory
    files = os.listdir(directory)

    # Filter out non-image files
    image_files = [f for f in files if f.endswith('.jpg')]

    # Check if there are enough image files
    if len(image_files) < num_images:
        print(f"Not enough images in {directory}.")
        return

    # Select random images
    random_images = random.sample(image_files, num_images)

    # Display selected images
    fig, axs = plt.subplots(1, num_images, figsize=(15, 5))
    for i, image_file in enumerate(random_images):
        image_path = os.path.join(directory, image_file)
        image = Image.open(image_path)
        axs[i].imshow(image)
        axs[i].axis('off')
    plt.show()

# Define source directory
source_dir = '/content/Explainable-techniques/Assignment-7/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER'
target = 'leopard'
concepts = ["dotted","striped","banded"]
random_folder = 'random500_10'

# Display 3 images from the target folder in a row
print("Images from target folder:")
display_random_images(os.path.join(source_dir, target), 3)

# Display 3 images from each concept folder
for concept in concepts:
    print(f"Images from {concept} folder:")
    display_random_images(os.path.join(source_dir, concept), 3)

# Display 3 images from a random folder
print("Images from random folder:")
display_random_images(os.path.join(source_dir, random_folder), 3)


In [ ]:
sess = utils.create_session() # a tensorflow session

GRAPH_PATH = "/content/tcav/tcav/inception5h/tensorflow_inception_graph.pb"
LABEL_PATH = "/content/tcav/tcav/inception5h/imagenet_comp_graph_label_strings.txt"

mymodel = model.GoogleNetWrapper_public(sess,
                                        GRAPH_PATH,
                                        LABEL_PATH)

In [ ]:
act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=100)

In [ ]:
absl.logging.set_verbosity(0)
num_random_exp=10 # Recommend >10
mytcav = tcav.TCAV(sess,
                   target,
                   concepts,
                   bottlenecks,
                   act_generator,
                   alphas,
                   cav_dir=cav_dir,
                   num_random_exp=num_random_exp)#10)
print ('Started')
results = mytcav.run(run_parallel=False)
print ('Completed')

In [ ]:
utils_plot.plot_results(results, num_random_exp=num_random_exp)